# Title                   

_Brief and informative, gives some idea of your topic area_  

## Introduction            
According to the Oxford English Dictionary (OED), the word underrated is defined as "Rated or valued too low." On the other hand, Merriam-Webster defines the term to be describing something that is "Not recognized for its true worth; undervalued." These definitions, when analyzed in the context of movies, could be interpreted to mean two different things: while the OED definition could be interpreted to describe a movie that is rated lower than it deserves, Merriam-Webster's definition could be taken to refer to movies that are simply not recognised to be good by a large enough group of people. For this project, we took Merriam-Webster's definition and used tools in data analytics to create the ultimate list of underrated movies across genres, regions, and decades. Our main challenege was to determine what it means for a movie to have low enough votes and high enough ratings to be 'underrated', and so our first step for this project was to use some basic statistical tools to determine a 'good' rating and a 'low' number of votes. 

## Database Design 

_Discuss how the database is structured, and how you have used this information to design your querying strategy. Discuss directly how this approach relates to the topic you decided to explore._

In [105]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/imdb_subset.db")
c = conn.cursor()


query = """

    SELECT 
    tb.titleId,tb.titleType,tb.primaryTitle,tb.originalTitle,tb.isAdult,tb.startYear,tb.endYear,tb.runtimeMinutes,
    r.averageRating,r.numVotes, 
    ta.region,ta.language,g.genre 
    FROM title_basics AS tb JOIN genres AS g ON tb.titleId = g.titleId 
    JOIN ratings AS r ON tb.titleId = r.titleId 
    JOIN title_akas AS ta ON tb.titleId = ta.titleId
    """


c.execute(query)


results = c.fetchall()

columns = ['titleId', 'titleType','primaryTitle', "originalTitle", "isAdult", "startyear", "endYear", 'runtimeMinutes', "averageRating","numVotes","region","language","genre"]


df = pd.DataFrame(results, columns=columns)
df.to_sql('joined_table', conn, if_exists='replace', index=False)
df


,titleId,titleType,primaryTitle,originalTitle,isAdult,startyear,endYear,runtimeMinutes,averageRating,numVotes,region,language,genre
0,tt0068494,movie,Domo Arigato,Domo Arigato,0,1990,None,91.0,4.7,21,None,None,Drama
1,tt0068494,movie,Domo Arigato,Domo Arigato,0,1990,None,91.0,4.7,21,None,None,Drama
2,tt0068494,movie,Domo Arigato,Domo Arigato,0,1990,None,91.0,4.7,21,US,None,Drama
3,tt0068494,movie,Domo Arigato,Domo Arigato,0,1990,None,91.0,4.7,21,US,None,Drama
4,tt0075745,movie,Beyond Reason,Beyond Reason,0,1985,None,88.0,4.8,82,None,None,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30991,tt9106768,movie,Michael Jackson: Smooth Criminal (II),Michael Jackson: Smooth Criminal (II),0,1988,None,48.0,8.5,654,US,None,Fantasy
30992,tt9441020,movie,Junkyard Boyz,Junkyard Boyz,0,1995,None,90.0,9.3,22,None,None,Drama
30993,tt9441020,movie,Junkyard Boyz,Junkyard Boyz,0,1995,None,90.0,9.3,22,GB,None,Drama
30994,tt9441020,movie,Junkyard Boyz,Junkyard Boyz,0,1995,None,90.0,9.3,22,TH,en,Drama


## Data Exploration        

_Use a blend of descriptive statistics and data visualizations to explore the subset. Include code blocks. Discuss potential areas for deeper analysis based on the data._

Our first goal, before we could move on to our main objective of classifying underrated movies for this project, was to determine a single number above which a movie could be classified as 'high-rated' as well as a single number below which a movie could be considered to have 'low' votes. To do this, we first looked at the median number of votes (so the result would not be skewed by movies like Shawshank Redemption!) and median number of ratings across movies in the IMDB dataset.

In [122]:
median_avgRating = """
WITH OrderedRatings AS (
    SELECT 
        averageRating,
        ROW_NUMBER() OVER (ORDER BY averageRating) AS row_num
    FROM 
        joined_table
),
CountedRatings AS (
    SELECT COUNT(*) AS total_count FROM OrderedRatings
)
SELECT AVG(averageRating) AS median
FROM OrderedRatings
WHERE row_num IN (
    (SELECT CAST((total_count + 1) / 2 AS INTEGER) FROM CountedRatings),
    (SELECT CAST((total_count + 2) / 2 AS INTEGER) FROM CountedRatings)
);
"""

median_rating = c.execute(median_avgRating).fetchone()
print(f"Median Rating: {median_rating[0]}")

Median Rating: 5.9


In [128]:
median_votes = """
WITH orderedvotes AS (
    SELECT 
        numVotes,
        ROW_NUMBER() OVER (ORDER BY numVotes) AS row_num
    FROM 
        joined_table
),
Countedvotes AS (
    SELECT COUNT(*) AS total_count FROM orderedvotes
)
SELECT AVG(numVotes) AS median
FROM orderedvotes
WHERE row_num IN (
    (SELECT CAST((total_count + 1) / 2 AS INTEGER) FROM Countedvotes),
    (SELECT CAST((total_count + 2) / 2 AS INTEGER) FROM Countedvotes)
);
"""

median_rating = c.execute(median_query).fetchone()
print(f"Median Rating: {median_rating[0]}")

Median Rating: 5.9


From executing these queries, we see that the median ratings of movies in IMDB are 5.9, whereas the median number of votes is 650. Based on these results, and answers from the IMDB FAQ that claimed that a good rating for a movie was about an average of 7, as well as various reddit discussion posts of numerous people claiming that they would not watch a movie with less than anywhere from 800-5000 votes, we did some averaging and decided to define an underrated movie for the context of this analysis as any above an average rating of 7.5, and below 10,000 votes. 

Given this, we first looked at top 10 underrated movies across all categories:

In [144]:
#Top ten underrated movies across all categories 
top_ten_underrated = c.execute('''
    SELECT numVotes, averageRating, primaryTitle
    FROM joined_table
    WHERE numVotes < 10000 AND averageRating > 7.5 GROUP BY primaryTitle
    ORDER BY numVotes DESC, averageRating DESC
    LIMIT 10
''').fetchall()
pd.DataFrame(top_ten_underrated)

,0,1,2
0,2828,8.2,Streetwise
1,2442,7.6,Hands on a Hardbody: The Documentary
2,2334,7.6,Kadhal Kottai
3,2209,7.6,The Endless Summer 2
4,1185,7.7,Divine Trash
5,994,7.8,American Dream
6,944,7.6,Gardish
7,654,8.5,Michael Jackson: Smooth Criminal (II)
8,622,8.9,PNYC: Portishead - Roseland New York
9,595,7.6,Message to Love: The Isle of Wight Festival


In [188]:
decades = range(1890, 2030, 10)
decade_dfs = {}
for decade in decades:
    query = f"""
    SELECT 
        primaryTitle,
        averageRating,
        numVotes,
        startYear
    FROM joined_table
    WHERE 
        startYear >= {decade} AND startYear < {decade + 10} AND
        numVotes < 10000 AND 
        averageRating > 7.5
    GROUP BY primaryTitle
    ORDER BY numVotes DESC, averageRating DESC
    LIMIT 10;
    """
    results = c.execute(query).fetchall()
    
    columns = ['Primary Title', 'Average Rating', 'Number of Votes', 'Start Year']
    decade_dfs[decade] = pd.DataFrame(results, columns=columns)

for decade, df in decade_dfs.items():
    print(f"Decade: {decade}s")
    print(df)
    print("\n")



Decade: 1890s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1900s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1910s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1920s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1930s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1940s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1950s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1960s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Index: []


Decade: 1970s
Empty DataFrame
Columns: [Primary Title, Average Rating, Number of Votes, Start Year]
Inde

## Uses of Python: Technical Components

_Fill in the table below with information about how you satisfied the technical requirements of the assignment._ 
    
| Component | Usage | Explanation | 
|:---|:---|:---|
| _leave this column as-is_ | _name the method(s)/ function(s) you used_ | _Use this area to explain what you were trying to achieve with this particular use of code and to describe why you chose to use this particular method or approach_ | 
| complex SQL query 1 | c.execute()| retrieve data from one or more tables |
| complex SQL query 2| |  | 
| complex SQL query 3 | | | 
| table join | | | 
| GROUP BY clause | | | 
| WHERE clause | | | 
| one of LIKE, HAVING, LIMIT, ORDER BY | | | 
| one SQLite function (e.g., count, max, min, etc.) | | | 
| dataviz 1 | | | 
| dataviz 2 | | | 
| dataviz 3 |  | | 


## Uses of Python: Reflection          

_Take a step back and analyze your own use of code. Provide some rationale for choices you’ve made. Considerations may include performance, human readability, code dependencies, and reproducibility._

## References              

_List all works cited in the data guide. Use proper APA format._